Mistral with and without SelfExtend

In [3]:
# not necessary to run flash attention if no SE
# GoogleColab: restart session after installing

!pip install flash-attn --no-build-isolation
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import json
import pandas as pd
import random

In [2]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import accelerate

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/comments_for_prompts.json') as f:
    reddit_comments = json.load(f)

with open('/content/drive/MyDrive/anes_questions.json') as q:
    questions = json.load(q)

anes = pd.read_csv('/content/drive/MyDrive/anes_timeseries_2020_csv_20220210.csv')
# Question had an option for free text answer, which is not useful for the analysis
# turned into negative value to not take it into account
anes['V201336'].replace(5, -7, inplace=True)

<ipython-input-4-319a210d25c0>:7: DtypeWarning: Columns (15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509) have mixed types. Specify dtype option on import or set low_memory=False.
  anes = pd.read_csv('/content/drive/MyDrive/anes_timeseries_2020_csv_20220210.csv')


In [5]:
questcode_topic = {"V201336": "abortion1",
              "V201342x": "abortion2",
              "V202332": "climate change1",
              "V202333": "climate change2",
              "V202337": "gun control1",
              "V202341x": "gun control2",
              "V202344x": "gun control3",
              "V202232": "immigration1",
              "V202233": "immigration2",
              "V202236x":"immigration3",
              "V202380x": "healthcare1"}

**Zero-Shot (without SelfExtend)**

In [ ]:
def prompt_zero(question, num_respondents):
    prompt_text = 'In the following you will be given a question followed several possible answer categories. Please \
    estimate how a representative sample of %d US-Americans would have answered the question in 2020 by providing the\
    number of respondents for each answer option in percent (output a dictionary object with answer category number as\
    key and answer frequency in percent as value. Only provide the number without the percentage sign).\
    Here is the questions: %s. '%(num_respondents, question)

    return prompt_text

In [ ]:
# without imputing number of responsents - sometimes confusing for model
def prompt_zero1(question):
    prompt_text = 'In the following you will be given a question followed several possible answer categories. Please \
    estimate how a representative sample of US-Americans would have answered the question in 2020 by providing the\
    number of respondents for each answer option in percent (output a dictionary object with answer category number as\
    key and answer frequency in percent as value. Only provide the number without the percentage sign).\
    Here is the questions: %s. '%(question)

    return prompt_text

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", torch_dtype=torch.float16, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
runs = 5
mistral_ans = {}

for i, question in enumerate(questions):
    #original survey valid answers
    valid_ans = anes[anes[question] > 0]
    survey_ans = len(valid_ans)

    answers_list = []

    for run in range(runs):

        #prompt for question (question + total answers)
        prompt_text = prompt_zero1(questions[question])

        prompt_postfix = "How would the representative sample look like? This is the frequency table: "
        prompt = prompt_text + prompt_postfix
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

        tokens = model.generate(input_ids, max_new_tokens=60)
        answer = tokenizer.decode(tokens[0].tolist()[input_ids.shape[1]:], skip_special_tokens=True)
        answers_list.append(answer)

    print("Iteration:", i+1, "/", len(questions))
    mistral_ans[question] = answers_list

mistral_ans

# Save to JSON
mistral_zero_ans = json.dumps(mistral_ans)
with open("answers_mistral_zero.json", "w") as file:
    file.write(mistral_zero_ans)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

Iteration: 1 / 11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

Iteration: 2 / 11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

...

Iteration: 9 / 11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

Iteration: 10 / 11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Iteration: 11 / 11


In [ ]:
# Save to JSON
import json
mistral_zero_ans = json.dumps(mistral_ans)
with open("answers_mistral_zero.json", "w") as file:
    file.write(mistral_zero_ans)

**Few Shot (with and without SelfExtend)**

In [ ]:
def prompt_comments(topic, discussion, question, num_respondents, num_comments = 100):
    headline = discussion[0][10:-1]

    shuffled_comments = discussion[1:]
    random.shuffle(shuffled_comments)
    comments = shuffled_comments[:num_comments]

    prompt_text = 'This is discussion on %s was held on Reddit in 2020. They are comments to a news article named "%s". \
    The comments score represents the net value of upvotes and downvotes. Afterwards you will be given a question followed \
    several possible answer categories. \
    Comments: %s.   \
    Please estimate how a representative sample of %d US-Americans would have answered the following question in 2020. Provide the \
    number of respondents for each answer option in percent (output a dictionary object with answer category number as \
    key and answer frequency in percent as value. Only provide the number without the percentage sign). \
    Here are the question and categories: %s. '%(topic, headline, comments, num_respondents, question)

    return prompt_text

In [6]:
def prompt_comments_strictly(topic, discussion, question, num_respondents, num_comments = 100):
    headline = discussion[0][10:-1]

    shuffled_comments = discussion[1:]
    random.shuffle(shuffled_comments)
    comments = shuffled_comments[:num_comments]

    prompt_text = 'This is discussion on %s was held on Reddit in 2020. They are comments to a news article named "%s". \
    The comments score represents the net value of upvotes and downvotes. Afterwards you will be given a question followed \
    several possible answer categories.\
    Comments: %s.   \
    Please estimate how a representative sample of %d US-Americans would have answered the following question in 2020.\
    Base yourself strictly in the comments provided. Output the number of respondents for each answer option in percent\
    (output a dictionary object with answer category number as key and answer frequency in percent as value.\
    Only provide the number without the percentage sign).\
    Here are the question and categories: %s'%(topic, headline, comments, num_respondents, question)

    return prompt_text

In [7]:
!git clone https://github.com/datamllab/LongLM.git

Cloning into 'LongLM'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 134 (delta 48), reused 53 (delta 35), pack-reused 61
Receiving objects: 100% (134/134), 882.08 KiB | 23.21 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
# !pip install transformers==4.36.2

In [ ]:
import transformers
transformers.__version__

'4.38.2'

In [8]:
import sys
sys.path.append('/content/LongLM')  # Add the directory to the system path

In [10]:
import accelerate
from self_extend_patch import Mistral
from self_extend_patch import selfextend_flash_attn

import warnings
warnings.filterwarnings("ignore")

import torch
import accelerate
import json
from transformers.models.llama.modeling_llama import LlamaAttention
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

import SelfExtend

In [11]:
runs = 5
num_comments = 100

In [13]:
mistral_ans_comments = {}
mistral_ans_comments_SE = {}

window_size = 1024
group_size = 32
use_flash = True

model_name = 'mistralai/Mistral-7B-Instruct-v0.1'

# Mistral with no SE
config = AutoConfig.from_pretrained(model_name)
config.sliding_window = None
model = AutoModelForCausalLM.from_pretrained(model_name, config=config, cache_dir='/data/HyJ',
                                                     device_map="auto", torch_dtype=torch.bfloat16,
                                                     use_flash_attention_2=use_flash)


tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/data/HyJ')
model.eval()

#"questions" object has all questions
for question in questions:
    answers_list = []
    answers_list_SE = []

    for run in range(runs):
        print(f"Iteration {run + 1}/{runs}")


        #original survey valid answers
        valid_ans = anes[anes[question] > 0]
        survey_ans = len(valid_ans)
        topic = questcode_topic[question][:-1]

        #prompt for question (question + total answers)
        prompt_text = prompt_comments_strictly(topic,
                                      reddit_comments[topic],
                                      questions[question],
                                      survey_ans,
                                      num_comments = num_comments)

        prompt_postfix = "How would the representative sample look like? This is the frequency table: "
        prompt = prompt_text + prompt_postfix


        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

        tokens = model.generate(input_ids, max_new_tokens=70)
        answer = tokenizer.decode(tokens[0].tolist()[input_ids.shape[1]:], skip_special_tokens=True)
        # answer = answer.replace("\n", "\\n")
        # answer= f"{model_name}:\n     [ {answer} ]"
        mistral_ans_comments[question] = answer
        answers_list.append(answer)

        #print("=========="*2 + "**SelfExtend**" + "=========="*2)

        # SelfExtend
        SelfExtend.apply(model, group_size, window_size, enable_flash_attention=use_flash)

        prompt_text = prompt_comments_strictly(topic,
                                  reddit_comments[topic],
                                  questions[question],
                                  survey_ans,
                                  num_comments = num_comments)

        prompt_postfix = "How would the representative sample look like? This is the frequency table: "
        prompt = prompt_text + prompt_postfix


        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

        tokens = model.generate(input_ids, max_new_tokens=60)
        answer = tokenizer.decode(tokens[0].tolist()[input_ids.shape[1]:], skip_special_tokens=True)
        answers_list_SE.append(answer)

    mistral_ans_comments[question] = answers_list
    mistral_ans_comments_SE[question] = answers_list_SE


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Iteration 1/5


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Iteration 2/5


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Iteration 4/5


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Iteration 5/5


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [14]:
# Save to JSON files
mistral_few_ans = json.dumps(mistral_ans_comments)
with open("mistral_few_strict.json", "w") as file:
    file.write(mistral_few_ans)

mistralSE_few_ans = json.dumps(mistral_ans_comments_SE)
with open("mistralSE_few_strict.json", "w") as file:
    file.write(mistralSE_few_ans)


**Test for SelfExtend: Passkey Example**

In [15]:
import warnings
warnings.filterwarnings("ignore")

import torch
import json
from transformers.models.llama.modeling_llama import LlamaAttention
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

import SelfExtend


window_size = 1024
group_size = 32
use_flash = True

model_lists = ['google/gemma-7b-it', 'meta-llama/Llama-2-7b-chat-hf', 'mistralai/Mistral-7B-Instruct-v0.1', ]

for model_name in model_lists[2:3]:
    if 'Mistral' in model_name:
        # Disable Mistral's sliding window
        config = AutoConfig.from_pretrained(model_name)
        config.sliding_window = None
        model = AutoModelForCausalLM.from_pretrained(model_name, config=config, cache_dir='/data/HyJ',
                                                     device_map="auto", torch_dtype=torch.bfloat16,
                                                     use_flash_attention_2=use_flash)
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir='/data/HyJ', device_map="auto", torch_dtype=torch.bfloat16, use_flash_attention_2=use_flash)

    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/data/HyJ')
    model.eval()
    file_name = "/content/LongLM/passkey_examples.jsonl"
    #for line in open("passkey_examples_5k.jsonl", "r"):
    #for line in open("passkey_examples_10k.jsonl", "r"):
    for line in open(file_name, "r"):
        example = json.loads(line)
        prompt_postfix = "What is the pass key? The pass key is "
        prompt = example["input"] + prompt_postfix
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        print( "-----------------------------------" )
        print( f"#Tokens of Prompt:", input_ids.shape[1], end=" " )
        print( "Passkey target:", example["target"] )

        tokens = model.generate(input_ids, max_new_tokens=len(example["target"]))
        answer = prompt_postfix + tokenizer.decode(tokens[0].tolist()[input_ids.shape[1]:], skip_special_tokens=True)
        answer = answer.replace("\n", "\\n")
        answer= f"{model_name}:\n     [ {answer} ]"
        print( answer )
        print( "-----------------------------------\n" )


    print("=========="*2 + "**SelfExtend**" + "=========="*2)
    SelfExtend.apply(model, group_size, window_size, enable_flash_attention=use_flash)
    for line in open(file_name, "r"):
        example = json.loads(line)
        prompt_postfix = "What is the pass key? The pass key is "
        prompt = example["input"] + prompt_postfix
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        print( f"#Tokens of Prompt:", input_ids.shape[1], end=" " )
        print( "Passkey target:", example["target"] )

        tokens = model.generate(input_ids, max_new_tokens=len(example["target"]))
        answer = prompt_postfix + tokenizer.decode(tokens[0].tolist()[input_ids.shape[1]:], skip_special_tokens=True)
        answer = answer.replace("\n", "\\n")
        answer= f"SelfExtended-{model_name}:\n     [ {answer} ]"
        print( answer )
        print( "-----------------------------------\n" )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 